# AUPIMO

Basic usage of the metric AUPIMO (pronounced "a-u-pee-mo").


# What is AUPIMO?

The `Area Under the Per-Image Overlap [curve]` (AUPIMO) is a metric of recall (higher is better) designed for visual anomaly detection.

Inspired by the [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) and [PRO](https://link.springer.com/article/10.1007/s11263-020-01400-4) curves, 

> AUPIMO is the area under a curve of True Positive Rate (TPR or _recall_) as a function of False Positive Rate (FPR) restricted to a fixed range. 

But:
- the TPR (Y-axis) is *per-image* (1 image = 1 curve/score);
- the FPR (X-axis) considers the (average of) **normal** images only; 
- the FPR (X-axis) is in log scale and its range is [1e-5, 1e-4]\* (harder detection task!).

\* The score (the area under the curve) is normalized to be in [0, 1].

AUPIMO can be interpreted as

> average segmentation recall in an image given that the model (nearly) does not yield false positives in normal images.

References in the last cell.

![AUROC vs. AUPRO vs. AUPIMO](./roc_pro_pimo.svg)

# Setup

Install `anomalib` using `pip`.

In [ ]:
# TODO(jpcbertoldo): replace by `pip install anomalib` when AUPIMO is released  # noqa: TD003
%pip install ../..

Change the directory to have access to the datasets.

In [ ]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded
#   to this directory.
dataset_root = Path.cwd().parent.parent / "datasets" / "MVTec"

## Imports

In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator, PercentFormatter
from scipy import stats

from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.metrics import AUPIMO, Evaluator
from anomalib.models import Padim

In [ ]:
%matplotlib inline

## Data Module

We will use dataset Leather from MVTec AD. 

> See the notebooks below for more details on datamodules. 
> [github.com/openvinotoolkit/anomalib/tree/main/examples/notebooks/100_datamodules](https://github.com/openvinotoolkit/anomalib/tree/main/examples/notebooks/100_datamodules)

In [ ]:
datamodule = MVTec(
    root=dataset_root,
    category="leather",
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=8,
)

# Average AUPIMO (Basic)

The easiest way to use AUPIMO is by creating the metric and setting it as the test metric in a new Evaluator. The evaluator will be attached to the model in the next step.

By default, the average AUPIMO is calculated.

In [ ]:
aupimo = AUPIMO()
evaluator = Evaluator(test_metrics=aupimo)

## Model

We will use `PaDiM` (performance is not the best, but it is fast to train).

> See the notebooks below for more details on models. 
> [github.com/openvinotoolkit/anomalib/tree/main/examples/notebooks/200_models](https://github.com/openvinotoolkit/anomalib/tree/main/examples/notebooks/200_models)

Instantiate the model and add the `Evaluator` instance which we created in the previous step.

In [ ]:
model = Padim(
    # only use one layer to speed it up
    layers=["layer1"],
    n_features=64,
    backbone="resnet18",
    pre_trained=True,
    evaluator=evaluator,
)

In [ ]:
engine = Engine(
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=False,
)
engine.fit(datamodule=datamodule, model=model)

In [ ]:
# will output the AUPIMO score on the test set
engine.test(datamodule=datamodule, model=model)

# Individual AUPIMO Scores (Detailed)

AUPIMO assigns one recall score per anomalous image in the dataset.

It is possible to access each of the individual AUPIMO scores and look at the distribution.

Collect the predictions and the ground truth.

In [ ]:
predictions = engine.predict(dataloaders=datamodule.test_dataloader(), model=model, return_predictions=True)

Compute the AUPIMO scores.

In [ ]:
aupimo = AUPIMO(
    # with `False` all the values are returned in a dataclass
    return_average=False,
)

for batch in predictions:
    aupimo.update(batch)

In [ ]:
# `pimo_result` has the PIMO curves of each image
# `aupimo_result` has the AUPIMO values
#     i.e. their Area Under the Curve (AUC)
pimo_result, aupimo_result = aupimo.compute()

Check the outputs.

In [ ]:
# the `nan`s are the normal images; they do not
# have a score because recall is not defined for them
print(aupimo_result.aupimos)

## Statistics

Compute statistics of the AUPIMO scores.

In [ ]:
# ignore removing the `nan`s
isnan = torch.isnan(aupimo_result.aupimos)

print(f"MEAN\n{aupimo_result.aupimos[~isnan].mean().item()=}")
print(f"OTHER STATISTICS\n{stats.describe(aupimo_result.aupimos[~isnan])}")

## Plot

Visualize the distribution of the AUPIMO scores.

In [ ]:
fig, ax = plt.subplots()
ax.hist(aupimo_result.aupimos.numpy(), bins=np.linspace(0, 1, 11), edgecolor="black")
ax.set_ylabel("Count (number of images)")
ax.yaxis.set_major_locator(MaxNLocator(5, integer=True))
ax.set_xlim(0, 1)
ax.set_xlabel("AUPIMO [%]")
ax.xaxis.set_major_formatter(PercentFormatter(1))
ax.grid()
ax.set_title("AUPIMO distribution")
fig  # noqa: B018, RUF100

# Cite Us

AUPIMO was developed during Google Summer of Code 2023 (GSoC 2023) with the `anomalib` team from OpenVINO Toolkit.

Our work was accepted to the British Machine Vision Conference 2024 (BMVC 2024).

```bibtex
@misc{bertoldo2024aupimo,
      title={{AUPIMO: Redefining Visual Anomaly Detection Benchmarks with High Speed and Low Tolerance}}, 
      author={Joao P. C. Bertoldo and Dick Ameln and Ashwin Vaidya and Samet Akçay},
      year={2024},
      eprint={2401.01984},
      archivePrefix={arXiv},
      primaryClass={cs.CV},
      url={https://arxiv.org/abs/2401.01984}, 
}
```

Paper on arXiv: [arxiv.org/abs/2401.01984](https://arxiv.org/abs/2401.01984) (accepted to BMVC 2024)

Medium post: [medium.com/p/c653ac30e802](https://medium.com/p/c653ac30e802)

Official repository: [github.com/jpcbertoldo/aupimo](https://github.com/jpcbertoldo/aupimo) (numpy-only API and numba-accelerated versions available)

GSoC 2023 page: [summerofcode.withgoogle.com/archive/2023/projects/SPMopugd](https://summerofcode.withgoogle.com/archive/2023/projects/SPMopugd)